In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'


In [ ]:
%pwd
%cd ..
%cd src
%pwd

'/home/kyudan/AnomLLM/notebook'

In [ ]:
#Please edit the BASE_DIR on your situation.

BASE_DIR = "/home/kyudan/AnomLLM"

In [5]:
def make_localization_prompt(anomal_data):
        
    LOCALIZATION_PROMPT = f"""Detect the ranges of anomalies in this time series based on the x-axis coordinate, where the x-axis coordinate refers to the x value in (x, y) data. The patterns of anomalies are as follows.
    {anomal_data}\nList one by one, in JSON format. If there are no anomalies, answer with an empty list [].

    Output template:
    [{{"start": ..., "end": ...}}, {{"start": ..., "end": ...}}...]
    """
    return LOCALIZATION_PROMPT

In [ ]:
from prompt import time_series_to_str
from utils import view_base64_image, display_messages, collect_results
from data.synthetic import SyntheticDataset

data_name = 'point'
data_dir = f'/home/kyudan/AnomLLM/data/synthetic/{data_name}/eval/'
train_dir = f'/home/kyudan/AnomLLM/data/synthetic/{data_name}/train/'
eval_dataset = SyntheticDataset(data_dir)
eval_dataset.load()
train_dataset = SyntheticDataset(train_dir)
train_dataset.load()

Loaded dataset point with 400 series.
Loaded dataset point with 400 series.


In [7]:
len(eval_dataset)

400

In [8]:
range(len(eval_dataset))

range(0, 400)

In [9]:
from utils import plot_series_and_predictions
from utils import view_base64_image, display_messages, collect_results

directory = f'/home/kyudan/AnomLLM/results/synthetic/{data_name}'
series = []
anomaly_locations = []
for idx in range(len(eval_dataset)): #(0부터 399)
    series.append(eval_dataset[idx][1].numpy()) # eval dataset의 문제 series.
    anomaly_locations.append(eval_dataset[idx][0].numpy()) # eval dataset의 gt가 될 것.

    #raw_results = collect_results(directory, raw=True)



In [10]:
#raw_results = {k: v[idx] for k, v in raw_results.items()}

In [11]:
"""results = collect_results(directory)
results = {k: v[idx] for k, v in results.items()}

fig = plot_series_and_predictions(
    series=series, 
    gt_anomaly_intervals=anomaly_locations,
    anomalies=results
)"""

'results = collect_results(directory)\nresults = {k: v[idx] for k, v in results.items()}\n\nfig = plot_series_and_predictions(\n    series=series, \n    gt_anomaly_intervals=anomaly_locations,\n    anomalies=results\n)'

In [12]:
from utils import display_messages

#display_messages(raw_results['gpt-4o-mini (0shot-text)']['request']['messages'])

In [13]:
#raw_results['gpt-4o-mini (0shot-text)']

In [14]:
anomaly_locations[33].tolist()

[[[442.0, 497.0], [806.0, 811.0]]]

In [15]:
anomaly_locations


[array([[[485., 500.]]], dtype=float32),
 array([[[341., 474.]]], dtype=float32),
 array([[[233., 238.],
         [562., 602.]]], dtype=float32),
 array([[[897., 902.]]], dtype=float32),
 array([[[200., 205.],
         [524., 580.],
         [780., 818.]]], dtype=float32),
 array([[[200., 231.],
         [622., 673.]]], dtype=float32),
 array([[[271., 304.],
         [867., 886.]]], dtype=float32),
 array([], shape=(1, 0), dtype=float32),
 array([[[373., 399.]]], dtype=float32),
 array([[[315., 377.]]], dtype=float32),
 array([[[575., 612.]]], dtype=float32),
 array([[[974., 987.]]], dtype=float32),
 array([[[651., 678.]]], dtype=float32),
 array([[[200., 214.]]], dtype=float32),
 array([[[785., 839.]]], dtype=float32),
 array([[[592., 625.]]], dtype=float32),
 array([[[459., 472.]]], dtype=float32),
 array([[[661., 667.]]], dtype=float32),
 array([[[856., 861.]]], dtype=float32),
 array([[[377., 402.]]], dtype=float32),
 array([[[768., 773.]]], dtype=float32),
 array([], shape=(1, 0),

In [16]:
def create_gt_data(anomaly_locations)->list :
    gt_data = []
    for i in anomaly_locations:
        anomal = i[0]
        gt_data.append([[int(x) for x in sublist] for sublist in anomal])
    return gt_data

In [17]:
#raw_text = raw_results['gpt-4o-mini (0shot-text)']['request']['messages'][0]['content'][0]['text']
#data_part = raw_text.split("\n\n")[0]
#data_part_list = data_part.
import numpy as np
def create_idx_datalist(arr:np.ndarray)->list:
    """
    입력: array([[ 0.00000000e+00],
       [ 1.87381312e-01],
       [ 3.68124545e-01]]) 형태.
    출력: [(0, '0.0'),
            (1, '0.19'),
            (2, '0.37')] 형태.
    """
    idx_data_list  = []
    for i, data in enumerate(time_series_to_str(arr).split(' ')):
        idx_data_list.append((i,data))
    return idx_data_list

def create_idx_data(list_:list) -> str:
    """
    입력: 
    [(0, '0.0'),
    (1, '0.19'),
    (2, '0.37')] 

    출력:
    '(0,0.0), (1,0.19), (2,0.37)' 형태
    """
    formatted_items = []
    for item in list_:
        formatted_item = f"({item[0]},{item[1]})"
        formatted_items.append(formatted_item)
    idx_data = ', '.join(formatted_items)
    return idx_data

In [18]:
create_idx_data(create_idx_datalist(series[0]))

'(0,0.0), (1,0.19), (2,0.37), (3,0.54), (4,0.68), (5,0.81), (6,0.9), (7,0.97), (8,1.0), (9,0.99), (10,0.95), (11,0.88), (12,0.77), (13,0.64), (14,0.48), (15,0.31), (16,0.13), (17,-0.06), (18,-0.25), (19,-0.43), (20,-0.59), (21,-0.73), (22,-0.84), (23,-0.93), (24,-0.98), (25,-1.0), (26,-0.98), (27,-0.93), (28,-0.84), (29,-0.73), (30,-0.59), (31,-0.43), (32,-0.25), (33,-0.06), (34,0.13), (35,0.31), (36,0.48), (37,0.64), (38,0.77), (39,0.88), (40,0.95), (41,0.99), (42,1.0), (43,0.97), (44,0.9), (45,0.81), (46,0.68), (47,0.54), (48,0.37), (49,0.19), (50,0.0), (51,-0.19), (52,-0.37), (53,-0.54), (54,-0.68), (55,-0.81), (56,-0.9), (57,-0.97), (58,-1.0), (59,-0.99), (60,-0.95), (61,-0.88), (62,-0.77), (63,-0.64), (64,-0.48), (65,-0.31), (66,-0.13), (67,0.06), (68,0.25), (69,0.43), (70,0.59), (71,0.73), (72,0.84), (73,0.93), (74,0.98), (75,1.0), (76,0.98), (77,0.93), (78,0.84), (79,0.73), (80,0.59), (81,0.43), (82,0.25), (83,0.06), (84,-0.13), (85,-0.31), (86,-0.48), (87,-0.64), (88,-0.77), (8

In [19]:
create_idx_datalist(series[0])

[(0, '0.0'),
 (1, '0.19'),
 (2, '0.37'),
 (3, '0.54'),
 (4, '0.68'),
 (5, '0.81'),
 (6, '0.9'),
 (7, '0.97'),
 (8, '1.0'),
 (9, '0.99'),
 (10, '0.95'),
 (11, '0.88'),
 (12, '0.77'),
 (13, '0.64'),
 (14, '0.48'),
 (15, '0.31'),
 (16, '0.13'),
 (17, '-0.06'),
 (18, '-0.25'),
 (19, '-0.43'),
 (20, '-0.59'),
 (21, '-0.73'),
 (22, '-0.84'),
 (23, '-0.93'),
 (24, '-0.98'),
 (25, '-1.0'),
 (26, '-0.98'),
 (27, '-0.93'),
 (28, '-0.84'),
 (29, '-0.73'),
 (30, '-0.59'),
 (31, '-0.43'),
 (32, '-0.25'),
 (33, '-0.06'),
 (34, '0.13'),
 (35, '0.31'),
 (36, '0.48'),
 (37, '0.64'),
 (38, '0.77'),
 (39, '0.88'),
 (40, '0.95'),
 (41, '0.99'),
 (42, '1.0'),
 (43, '0.97'),
 (44, '0.9'),
 (45, '0.81'),
 (46, '0.68'),
 (47, '0.54'),
 (48, '0.37'),
 (49, '0.19'),
 (50, '0.0'),
 (51, '-0.19'),
 (52, '-0.37'),
 (53, '-0.54'),
 (54, '-0.68'),
 (55, '-0.81'),
 (56, '-0.9'),
 (57, '-0.97'),
 (58, '-1.0'),
 (59, '-0.99'),
 (60, '-0.95'),
 (61, '-0.88'),
 (62, '-0.77'),
 (63, '-0.64'),
 (64, '-0.48'),
 (65, '-0.31'

In [20]:
gt = create_gt_data(anomaly_locations)
gt[4]

[[200, 205], [524, 580], [780, 818]]

In [21]:
def create_anomal_series(series_list, gt):
    result_list = []
    result = []
    for diff_anom in gt:
        # ground truth index는 1부터 1000이라 아래와 같이 indexing 해야 out of range error 안 뜸.
        for i in range(diff_anom[0]-1,diff_anom[1]):
            result_list.append(series_list[i])
        result.append(result_list)
        result_list = []
    
    return result # 0.19 0.37 0.54 이런 형태.

In [22]:
create_anomal_series(create_idx_datalist(series[4]), gt[4])

[[(199, '-0.19'),
  (200, '0.75'),
  (201, '0.04'),
  (202, '0.81'),
  (203, '-0.69'),
  (204, '-0.85')],
 [(523, '-0.93'),
  (524, '-0.03'),
  (525, '0.19'),
  (526, '-0.02'),
  (527, '-1.03'),
  (528, '-0.04'),
  (529, '-0.65'),
  (530, '0.33'),
  (531, '0.18'),
  (532, '-0.47'),
  (533, '-0.26'),
  (534, '-0.53'),
  (535, '-0.03'),
  (536, '0.48'),
  (537, '-0.49'),
  (538, '0.25'),
  (539, '-0.27'),
  (540, '-0.4'),
  (541, '-0.05'),
  (542, '-0.52'),
  (543, '-0.28'),
  (544, '-0.6'),
  (545, '0.98'),
  (546, '0.02'),
  (547, '-0.35'),
  (548, '0.11'),
  (549, '-0.06'),
  (550, '-0.11'),
  (551, '0.31'),
  (552, '0.38'),
  (553, '-0.27'),
  (554, '-0.29'),
  (555, '-0.14'),
  (556, '-1.15'),
  (557, '-0.76'),
  (558, '0.68'),
  (559, '0.82'),
  (560, '-0.12'),
  (561, '0.29'),
  (562, '0.16'),
  (563, '1.54'),
  (564, '0.56'),
  (565, '-0.06'),
  (566, '-0.48'),
  (567, '-0.8'),
  (568, '0.1'),
  (569, '-0.38'),
  (570, '-0.71'),
  (571, '-0.32'),
  (572, '-0.54'),
  (573, '0.84')

In [23]:
def create_anomal_data_prompt(anomal_list):
    result = []
    
    for i, anomaly_group in enumerate(anomal_list, 1):
        result.append(f"anomaly data {i}:")
        
        values = [item[1] for item in anomaly_group]
        result.append(" ".join(values))
        
        if i < len(anomal_list):
            result.append("")
    
    return "\n".join(result)

In [24]:
print(create_anomal_data_prompt(create_anomal_series(create_idx_datalist(series[4]), gt[4])))

anomaly data 1:
-0.19 0.75 0.04 0.81 -0.69 -0.85

anomaly data 2:
-0.93 -0.03 0.19 -0.02 -1.03 -0.04 -0.65 0.33 0.18 -0.47 -0.26 -0.53 -0.03 0.48 -0.49 0.25 -0.27 -0.4 -0.05 -0.52 -0.28 -0.6 0.98 0.02 -0.35 0.11 -0.06 -0.11 0.31 0.38 -0.27 -0.29 -0.14 -1.15 -0.76 0.68 0.82 -0.12 0.29 0.16 1.54 0.56 -0.06 -0.48 -0.8 0.1 -0.38 -0.71 -0.32 -0.54 0.84 0.44 -0.0 0.74 0.04 -0.43 0.76

anomaly data 3:
0.73 0.27 -0.52 -0.1 -0.44 -0.69 0.46 0.95 -0.7 0.28 -0.33 -0.24 -0.3 -0.43 0.02 -0.42 0.14 -0.03 -0.12 -0.45 -0.29 0.38 0.25 -0.49 0.05 0.38 -0.83 0.27 -0.33 0.29 -0.38 -0.9 -0.81 0.02 0.13 -0.45 0.32 -0.83 -0.03


In [33]:
from utils import plot_series_and_predictions
from utils import view_base64_image, display_messages, collect_results
from prompt import LIMIT_PROMPT
directory = f'/home/kyudan/AnomLLM/results/synthetic/{data_name}'
series = []
anomaly_locations = []
for idx in range(len(eval_dataset)): #(0부터 399)
    series.append(eval_dataset[idx][1].numpy()) # eval dataset의 문제 series.
    anomaly_locations.append(eval_dataset[idx][0].numpy()) # eval dataset의 gt가 될 것.

def create_localization_prompt(
        idx,
        series,
        anomaly_locations,
):
    gt = create_gt_data(anomaly_locations)
    #idx_data = create_idx_data(create_idx_datalist(series[idx]))
    #idx_list = create_idx_datalist(series[idx])
    anomal_text = create_anomal_series(create_idx_datalist(series[idx]), gt[idx]) #j번째 anomal
    
    messages = [
        {
            "role": "user",
            "content": str(time_series_to_str(series[idx])) 
                    + "\n\n" 
                    + LIMIT_PROMPT 
                    + make_localization_prompt(create_anomal_data_prompt(anomal_text))
        }
    ]
    
    return {
        "messages": messages,
        "temperature": 0.4,
        "stop": ["''''", " – –", "<|endoftext|>", "<|eot_id|>"]
    }


In [34]:
anomaly_locations

[array([[[485., 500.]]], dtype=float32),
 array([[[341., 474.]]], dtype=float32),
 array([[[233., 238.],
         [562., 602.]]], dtype=float32),
 array([[[897., 902.]]], dtype=float32),
 array([[[200., 205.],
         [524., 580.],
         [780., 818.]]], dtype=float32),
 array([[[200., 231.],
         [622., 673.]]], dtype=float32),
 array([[[271., 304.],
         [867., 886.]]], dtype=float32),
 array([], shape=(1, 0), dtype=float32),
 array([[[373., 399.]]], dtype=float32),
 array([[[315., 377.]]], dtype=float32),
 array([[[575., 612.]]], dtype=float32),
 array([[[974., 987.]]], dtype=float32),
 array([[[651., 678.]]], dtype=float32),
 array([[[200., 214.]]], dtype=float32),
 array([[[785., 839.]]], dtype=float32),
 array([[[592., 625.]]], dtype=float32),
 array([[[459., 472.]]], dtype=float32),
 array([[[661., 667.]]], dtype=float32),
 array([[[856., 861.]]], dtype=float32),
 array([[[377., 402.]]], dtype=float32),
 array([[[768., 773.]]], dtype=float32),
 array([], shape=(1, 0),

In [35]:
gt = create_gt_data(anomaly_locations)
gt[7]

[]

In [36]:
print(create_localization_prompt(7, series, anomaly_locations))

{'messages': [{'role': 'user', 'content': '0.0 0.19 0.37 0.54 0.68 0.81 0.9 0.97 1.0 0.99 0.95 0.88 0.77 0.64 0.48 0.31 0.13 -0.06 -0.25 -0.43 -0.59 -0.73 -0.84 -0.93 -0.98 -1.0 -0.98 -0.93 -0.84 -0.73 -0.59 -0.43 -0.25 -0.06 0.13 0.31 0.48 0.64 0.77 0.88 0.95 0.99 1.0 0.97 0.9 0.81 0.68 0.54 0.37 0.19 0.0 -0.19 -0.37 -0.54 -0.68 -0.81 -0.9 -0.97 -1.0 -0.99 -0.95 -0.88 -0.77 -0.64 -0.48 -0.31 -0.13 0.06 0.25 0.43 0.59 0.73 0.84 0.93 0.98 1.0 0.98 0.93 0.84 0.73 0.59 0.43 0.25 0.06 -0.13 -0.31 -0.48 -0.64 -0.77 -0.88 -0.95 -0.99 -1.0 -0.97 -0.9 -0.81 -0.68 -0.54 -0.37 -0.19 -0.0 0.19 0.37 0.54 0.68 0.81 0.9 0.97 1.0 0.99 0.95 0.88 0.77 0.64 0.48 0.31 0.13 -0.06 -0.25 -0.43 -0.59 -0.73 -0.84 -0.93 -0.98 -1.0 -0.98 -0.93 -0.84 -0.73 -0.59 -0.43 -0.25 -0.06 0.13 0.31 0.48 0.64 0.77 0.88 0.95 0.99 1.0 0.97 0.9 0.81 0.68 0.54 0.37 0.19 0.0 -0.19 -0.37 -0.54 -0.68 -0.81 -0.9 -0.97 -1.0 -0.99 -0.95 -0.88 -0.77 -0.64 -0.48 -0.31 -0.13 0.06 0.25 0.43 0.59 0.73 0.84 0.93 0.98 1.0 0.98 0.93 0.84 0

In [38]:
from openai_api import send_openai_request
def online_AD_with_retries(
    model_name: str,
    data_name: str,
    #request_func: callable,
    variant: str = "standard",
    num_retries: int = 4,
):
    import json
    import time
    import pickle
    import os
    from loguru import logger
    from data.synthetic import SyntheticDataset

    # Initialize dictionary to store results
    results = {}

    # Configure logger
    log_fn = f"logs/synthetic/{data_name}/{model_name}/" + variant + ".log"
    logger.add(log_fn, format="{time} {level} {message}", level="INFO")
    results_dir = f'/home/kyudan/AnomLLM/results/synthetic/{data_name}/{model_name}/'
    data_dir = f'/home/kyudan/AnomLLM/data/synthetic/{data_name}/eval/'
    train_dir = f'/home/kyudan/AnomLLM/data/synthetic/{data_name}/train/'
    jsonl_fn = os.path.join(results_dir, variant + '.jsonl')
    os.makedirs(results_dir, exist_ok=True)

    eval_dataset = SyntheticDataset(data_dir)
    eval_dataset.load()

    train_dataset = SyntheticDataset(train_dir)
    train_dataset.load()

    # Load existing results if jsonl file exists
    if os.path.exists(jsonl_fn):
        with open(jsonl_fn, 'r') as f:
            for line in f:
                entry = json.loads(line.strip())
                results[entry['custom_id']] = entry["response"]

    # Loop over image files
    for i in range(1, len(eval_dataset) + 1):
        custom_id = f"{data_name}_{model_name}_{variant}_{str(i).zfill(5)}"
        gt = create_gt_data(anomaly_locations)
        if len(gt[i-1]) == 0:
            logger.info(f"Skipping {custom_id} as it has no anomaly locations")
            continue
        # Skip already processed files
        if custom_id in results:
            continue
        
        # Perform anomaly detection with exponential backoff
        for attempt in range(num_retries):
            try:
                request = create_localization_prompt(i-1, series, anomaly_locations)
                response = send_openai_request(request, model_name)
                # Write the result to jsonl
                with open(jsonl_fn, 'a') as f:
                    json.dump({'custom_id': custom_id, 'request': request, 'response': response}, f)
                    f.write('\n')
                # If successful, break the retry loop
                break
            except Exception as e:
                if "503" in str(e):  # Server not up yet, sleep until the server is up again
                    while True:
                        logger.debug("503 error, sleep 30 seconds")
                        time.sleep(30)
                        try:
                            response = send_openai_request(request, model_name)
                            break
                        except Exception as e:
                            if "503" not in str(e):
                                break
                else:
                    logger.error(e)
                    # If an exception occurs, wait and then retry
                    wait_time = 2 ** (attempt + 3)
                    logger.debug(f"Attempt {attempt + 1} failed. Waiting for {wait_time} seconds before retrying...")
                    time.sleep(wait_time)
                    continue
        else:
            logger.error(f"Failed to process {custom_id} after {num_retries} attempts")

In [39]:
online_AD_with_retries('gemini-1.5-flash', 'point', 'localization (rawData)')

2025-03-30 23:15:33.567 | DEBUG    | gemini_api:send_gemini_request:93 - API key: ***********************************go6Y


Loaded dataset point with 400 series.
Loaded dataset point with 400 series.


2025-03-30 23:15:35.010 | DEBUG    | gemini_api:send_gemini_request:93 - API key: ***********************************go6Y
2025-03-30 23:15:36.589 | DEBUG    | gemini_api:send_gemini_request:93 - API key: ***********************************go6Y
2025-03-30 23:15:38.281 | DEBUG    | gemini_api:send_gemini_request:93 - API key: ***********************************go6Y
2025-03-30 23:15:39.604 | DEBUG    | gemini_api:send_gemini_request:93 - API key: ***********************************go6Y
2025-03-30 23:15:41.316 | DEBUG    | gemini_api:send_gemini_request:93 - API key: ***********************************go6Y
2025-03-30 23:15:42.954 | DEBUG    | gemini_api:send_gemini_request:93 - API key: ***********************************go6Y
2025-03-30 23:15:44.501 | INFO     | __main__:online_AD_with_retries:46 - Skipping point_gemini-1.5-flash_localization (rawData)_00008 as it has no anomaly locations
2025-03-30 23:15:44.506 | DEBUG    | gemini_api:send_gemini_request:93 - API key: ********************